# 05-2 교차 검증과 그리드 서치

## 검증 세트

### 검증 세트

In [2]:
# 데이터를 불러옴
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [3]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [4]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [5]:
# 모델 훈련을 위한 훈련 세트 sub_input, sub_target
# 하이퍼파라미터 튜닝을 위한 검증 세트 val_input, val_target
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

In [6]:
# 모델을 만들고 평가
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


## 교차 검증

### 교차 검증

In [7]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
print(scores) # 모델을 훈련하는 시간, 검증하는 시간, 교차 검증 점수

{'fit_time': array([0.009094  , 0.00635433, 0.00601292, 0.0060246 , 0.00647378]), 'score_time': array([0.00074506, 0.00050402, 0.00048161, 0.00052834, 0.00054264]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [8]:
import numpy as np

# 교차 검증의 최종 점수
print(np.mean(scores['test_score']))

0.855300214703487


### 분할기를 사용한 교차 검증

In [9]:
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [10]:
# 만약 훈련 세트를 석은 후 10-폴드 교차 검증을 수행
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


## 하이퍼파라미터 튜닝

### 그리드 서치

In [12]:
# GridSearchCV 클래스를 임포트하고 탐색할 매개변수와 탐색할 값의 리스트를 딕셔너리로 만듦
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [13]:
# GridSearchCV 클래스에 탐색 대상 모델과 params 변수를 전달하여 그리드 서치 객체를 만들고
# 결정 트리 클래스의 객체를 생성해서 바로 전달
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

In [14]:
# fit() 메소드를 호출하면 
# 그리드 서치 객체는 결정 트리 모델 min_impurity_decrease 값을 바꿔가면서 총 5번을 실행함
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [15]:
# 훈련이 끝나면 25개의 모델 중에서 검증 점수가 가장 높은 모델의 매개변수 조합으로
# 전체 훈련 세트에서 자동으로 다시 모델을 훈련한 후 gs 객체의 best_estimator_ 속성에 저장
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [16]:
# 그리드 서치로 찾은 최적의 매개변수를 best_params_ 속성에 저장
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [17]:
# 각 매개변수에서 수행한 교차 검증의 평균 점수는 cv_results_속성의 mean_test_score 키에 저장
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [18]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001), # 0.0001에서 시작하여 0.001이 될 때까지 0.0001을 계속 더한 배열
          'max_depth': range(5, 20, 1), # 5에서 20까지 1씩 증가하면서 15개의 값을 만듦
          'min_samples_split': range(2, 100, 10) # 2에서 100까지 10씩 증가하면서 10개의 값을 만듦
          }

In [19]:
#교차 검증 횟수는 9 X 15 X 10 = 1350개
# 기본 50폴드 교차 검증을 수행하므로 만들어지는 모델 수는 6750개
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [19]:
# 최상의 매개변수 조합
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [20]:
# 최상의 교차 검증 점수
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


### 확률 분포 선택

In [22]:
from scipy.stats import uniform, randint

In [23]:
# 0 ~ 10 사이의 범위를 갖는 randint 객체를 만들고 1000개를 샘플링해서 각 숫자의 개수를 세어봄
# 0에서 9까지의 숫자가 어느 정도 고르게 추출됨
rgen = randint(0, 10)
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 98,  94,  94, 109, 104,  87,  94, 109, 107, 104]))

In [24]:
# 0 ~ 1 사이의 범위를 갖는 uniform 객체를 만들고 10개의 실수를 추출
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.17234201, 0.99128101, 0.21528886, 0.77747088, 0.34898384,
       0.05554056, 0.92572525, 0.92655294, 0.53767127, 0.67294402])

### 랜덤 서치

In [25]:
# 매개변수의 딕셔너리 생성
params = {'min_impurity_decrease': uniform(0.0001, 0.001), # 0.0001에서 0.001 사이의 실숫값을 샘플링
          'max_depth': randint(20, 50), # 20에서 50 사이의 정수를 샘플링
          'min_samples_split': randint(2, 25), # 2에서 25 사이의 정수를 샘플링
          'min_samples_leaf': randint(1, 25), # 1에서 25 사이의 정수를 샘플링
          }

In [26]:
from sklearn.model_selection import RandomizedSearchCV

# params에 정의된 매개변수 범위에서 총 100번을 샘플링하여 교차 검증을 수행
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ff3c5b35f90>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ff3c51dde10>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ff3c5205d50>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ff3c5b350d0>},
                   random_state=42)

In [30]:
# 최적의 매개변수 조합
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [28]:
# 최적의 매개변수로 찾은 검증 세트의 점수인 최고 교차 검증 점수
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [29]:
dt = gs.best_estimator_
# 최종 모델로 테스트 세트의 성능을 확인
print(dt.score(test_input, test_target))

0.86


### +) 확인 문제

In [31]:
# RamdomizedSearchCV 예제에서 DecisionTreeClassifier 클래스에 splitter='random' 매개변수를 추가하고 다시 훈련
gs = RandomizedSearchCV(DecisionTreeClassifier(splitter='random', random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42,
                                                    splitter='random'),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ff3c5b35f90>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ff3c51dde10>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ff3c5205d50>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ff3c5b350d0>},
                   random_state=42)

In [32]:
# # 최적의 매개변수 조합
print(gs.best_params_)
# 최고 교차 검증 점수
print(np.max(gs.cv_results_['mean_test_score']))

dt = gs.best_estimator_
# 최종 모델로 테스트 세트의 성능을 확인
print(dt.score(test_input, test_target))

{'max_depth': 43, 'min_impurity_decrease': 0.00011407982271508446, 'min_samples_leaf': 19, 'min_samples_split': 18}
0.8458726956392981
0.786923076923077
